In [48]:
import pandas as pd
import numpy as np
import scipy

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

In [49]:
aucs = pd.read_csv("AUCs.txt", sep = '\t', header = 0)[["C4.5","C4.5+m","C4.5+cf","C4.5+m+cf"]]

In [50]:
aucs.head()

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,0.763,0.768,0.771,0.798
1,0.599,0.591,0.590,0.569
2,0.954,0.971,0.968,0.967
3,0.628,0.661,0.654,0.657
4,0.882,0.888,0.886,0.898


In [51]:
corr_data = []

for i, lhs_column in enumerate(aucs.columns):
    for j, rhs_column in enumerate(aucs.columns):
        if i >= j:
            continue
        
        corr, p = scipy.stats.wilcoxon(aucs[lhs_column], aucs[rhs_column])
        corr_data.append([lhs_column, rhs_column, corr, p])

In [52]:
aucs_correlation = pd.DataFrame.from_records(corr_data)
aucs_correlation.columns = ['model_A', 'model_B', 'corr', 'p']

In [53]:
aucs_correlation

,model_A,model_B,corr,p
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
5,C4.5+cf,C4.5+m+cf,10.0,0.022909


In [54]:
(aucs_correlation.p < 0.05).value_counts()

True     4
False    2
Name: p, dtype: int64

In [55]:
reject, p_corrected, a1, a2 = multipletests(aucs_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'holm')

In [56]:
aucs_correlation['p_corrected'] = p_corrected
aucs_correlation['reject'] = reject

In [57]:
aucs_correlation

,model_A,model_B,corr,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.091636,False


In [58]:
aucs_correlation.reject.value_counts()

False    6
Name: reject, dtype: int64

In [59]:
reject, p_corrected, a1, a2 = multipletests(aucs_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [60]:
aucs_correlation['p_corrected'] = p_corrected
aucs_correlation['reject'] = reject

In [61]:
aucs_correlation

,model_A,model_B,corr,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.045818,True
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,17.0,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.045818,True


In [62]:
aucs_correlation.reject.value_counts()

True     3
False    3
Name: reject, dtype: int64